In [4]:
import pandas as pd
import numpy as np
import mysql.connector
import seaborn as sns
import matplotlib.pyplot as plt
from googletrans import Translator
from folium import Map,CircleMarker,Marker

In [5]:

c=pd.read_csv("gender_age_train.csv")
d=pd.read_csv("phone_brand_device_model.csv")
events_data=pd.read_csv('events_data.csv',dtype={'device_id': 'string'})

In [6]:
print("user_data_missing:",c.device_id.isnull().sum())
print("device_data_missing:",d.device_id.isnull().sum())
print("events_data_missing:",events_data.device_id.isnull().sum())
len(events_data)

user_data_missing: 0
device_data_missing: 0
events_data_missing: 453


3252950

### DEVICES DATA

In [7]:
phn_brnd=pd.DataFrame(d['phone_brand'].drop_duplicates()).reset_index()
phn_brnd.drop(['index'],axis=1,inplace=True)

In [8]:
translator = Translator()
l=[]
for i  in range(len(phn_brnd)):
    translations = translator.translate([phn_brnd['phone_brand'][i]], dest='en')
    for translation in translations:
        l.append(translation.text)

In [9]:
phn_brnd['eng_brand']=l
mob_devices=pd.merge(d,phn_brnd,how='left',left_on='phone_brand',right_on='phone_brand')
mob_devices['eng_brand'].drop_duplicates()

0              vivo
1            Xiaomi
2              OPPO
3           Samsung
11          Coolpad
            ...    
64701    Hengyufeng
66501          Oaks
71650       Siemens
79470         Oledi
79851          PPTV
Name: eng_brand, Length: 116, dtype: object

##### some brands looks like are not real

In [10]:
print(mob_devices[mob_devices['eng_brand']=='Quality rice'].head(2),"\n",len(mob_devices[mob_devices['eng_brand']=='Quality rice']))
print(mob_devices[mob_devices['eng_brand']=='Big Coke'].head(2),"\n",len(mob_devices[mob_devices['eng_brand']=='Big Coke']))
print(mob_devices[mob_devices['eng_brand']=='black rice'].head(2),"\n",len(mob_devices[mob_devices['eng_brand']=='black rice']))
print(mob_devices[mob_devices['eng_brand']=='Fresh rice'].head(2),"\n",len(mob_devices[mob_devices['eng_brand']=='Fresh rice']))
print(mob_devices[mob_devices['eng_brand']=='Bacardi'].head(2),"\n",len(mob_devices[mob_devices['eng_brand']=='Bacardi']))
print(mob_devices[mob_devices['eng_brand']=="Cube Rubik's Cube"].head(2),"\n",len(mob_devices[mob_devices['eng_brand']=="Cube Rubik's Cube"]))

     Unnamed: 0            device_id phone_brand device_model     eng_brand
123         123  -942277992798295690          优米            3  Quality rice
132         132  1251552303715560512          优米           R1  Quality rice 
 243
Empty DataFrame
Columns: [Unnamed: 0, device_id, phone_brand, device_model, eng_brand]
Index: [] 
 0
      Unnamed: 0            device_id phone_brand device_model   eng_brand
2861        2861 -1145583104199299037          黑米           A1  black rice
4213        4213 -1576379705054644712          黑米           A1  black rice 
 47
       Unnamed: 0            device_id phone_brand device_model   eng_brand
28927       28927 -4478793803614584475          鲜米         s800  Fresh rice
77154       77154  1751236410718403505          鲜米         s800  Fresh rice 
 2
Empty DataFrame
Columns: [Unnamed: 0, device_id, phone_brand, device_model, eng_brand]
Index: [] 
 0
Empty DataFrame
Columns: [Unnamed: 0, device_id, phone_brand, device_model, eng_brand]
Index: [] 
 0


### EVENTS DATA

In [11]:
events_data.isnull().sum()


event_id       0
device_id    453
timestamp      0
longitude    423
latitude     423
city           0
state        377
dtype: int64

### Missing value treatment
- Missing device IDs are filled in with device ids from other same lat-longs available
- States are filled in with states from other same cities available.
- Missing lat longs are filled in with lat longs from other same deviceIDs available

In [12]:
nonunique=pd.DataFrame(events_data.groupby(['longitude','latitude'])['device_id'].nunique().sort_values(ascending=False))
nonunique=nonunique[nonunique['device_id']>1].reset_index()

In [13]:
for i in range(len(nonunique)):
    a="b"+str(i)
    a=events_data[events_data['device_id'].isnull()& (events_data['longitude'] ==nonunique['longitude'][i]) & (events_data['latitude'] ==nonunique['latitude'][i])][['latitude','longitude']]
    print(len(a+i))

0
0
0


In [14]:
#events_data['device_id']=events_data['device_id'].map(lambda x: '{:.0f}'.format(x)).astype(str)
events1=pd.DataFrame(events_data.groupby(['longitude','latitude'])['device_id'].apply(lambda S: S.mode()[0])).reset_index()
events_data_final=pd.merge(events_data,events1,how='left', on=['longitude','latitude'])
#events_data_final['device_id_x']=events_data_final['device_id_x'].map(lambda x: '{:.0f}'.format(x)).astype(str)
#events_data_final['device_id_y']=events_data_final['device_id_y'].map(lambda x: '{:.0f}'.format(x)).astype(str)
l=[]
for i in range(len(events_data_final)):
    if pd.isnull(events_data_final['device_id_x'][i]):
        l.append(events_data_final['device_id_y'][i])
    else:
        l.append(events_data_final['device_id_x'][i])

events_data_final['device_id']=l

In [ ]:
events_state=pd.DataFrame(events_data_final.groupby(['device_id'])['state'].apply(lambda S: S.mode()[0])).reset_index()
events_s2=pd.merge(events_data_final,events_state,how='left',left_on=['device_id'],right_on=['device_id'])

l=[]
for i in range(len(events_s2)):
    if pd.isnull(events_s2['state_x'][i]):
        l.append(events_s2['state_y'][i])
    else:
        l.append(events_s2['state_x'][i])
        
events_s2['state']=l

In [ ]:
events_latlong=pd.DataFrame(events_s2.groupby(['device_id'])['longitude'].apply(lambda S: S.mode()[0])).reset_index()
events_s2=pd.merge(events_s2,events_latlong,how='left',left_on=['device_id'],right_on=['device_id'])

l=[]
for i in range(len(events_s2)):
    if pd.isnull(events_s2['longitude_x'][i]):
        l.append(events_s2['longitude_y'][i])
    else:
        l.append(events_s2['longitude_x'][i])
        
events_s2['longitude']=l

In [ ]:
events_latlong1=pd.DataFrame(events_s2.groupby(['device_id'])['latitude'].apply(lambda S: S.mode()[0])).reset_index()
events_s2=pd.merge(events_s2,events_latlong1,how='left',left_on=['device_id'],right_on=['device_id'])
l=[]
for i in range(len(events_s2)):
    if pd.isnull(events_s2['latitude_x'][i]):
        l.append(events_s2['latitude_y'][i])
    else:
        l.append(events_s2['latitude_x'][i])
        
events_s2['latitude']=l
data1=events_s2[['latitude','longitude']]

In [ ]:
%matplotlib inline
#data1['latitude'].plot()
data1['longitude'].plot()

In [ ]:
%matplotlib inline
data1['latitude'].plot()

In [ ]:
data2=data1[(data1['longitude']< 69.3) | (data1['latitude']> 35.0)][['latitude','longitude']].drop_duplicates().reset_index()


In [ ]:
data2

In [ ]:
## events_s2

In [ ]:
import reverse_geocoder as rg
results=[]
m=[]
for i in range(len(data2)):
    results.append(rg.search((data2['latitude'][i],data2['longitude'][i])))
    m.append(results[i][0]['admin1'])

In [ ]:
data2['state_latlong']=m

In [ ]:
wrong_latlong=pd.merge(events_s2,data2, how="left",on=["latitude","longitude"])

In [ ]:
wrong_latlong.drop(['device_id_x','longitude_x', 'latitude_x','state_x', 'device_id_y','state_y','longitude_y','latitude_y',
                   'index'],axis=1,inplace=True)

In [ ]:
final_data=pd.DataFrame(wrong_latlong.groupby(['state'])['latitude'].apply(lambda S: S.mode()[0])).reset_index()
final_data=pd.merge(wrong_latlong,final_data,how='left',left_on=['state'],right_on=['state'])
l=[]
#m=[]
for i in range(len(final_data)):
    if pd.isnull(final_data['state_latlong'][i]):
        l.append(final_data['latitude_x'][i])
        #m.append(final_data['longitude_x'][i])
    else:
        l.append(final_data['latitude_y'][i])
        #m.append(final_data['longitude_y'][i])
        
final_data['latitude']=l
#final_data['longitude']=m
#data1=events_s2[['latitude','longitude']]

In [ ]:
final_data1=pd.DataFrame(final_data.groupby(['state'])['longitude'].apply(lambda S: S.mode()[0])).reset_index()
final_data=pd.merge(final_data,final_data1,how='left',left_on=['state'],right_on=['state'])
l=[]
#m=[]
for i in range(len(final_data)):
    if pd.isnull(final_data['state_latlong'][i]):
        l.append(final_data['longitude_x'][i])
        #m.append(final_data['longitude_x'][i])
    else:
        l.append(final_data['longitude_y'][i])
        #m.append(final_data['longitude_y'][i])
        
final_data['longitude']=l
#final_data['longitude']=m
#data1=events_s2[['latitude','longitude']]

In [ ]:
## events_s2[events_s2['state_latlong']!=events_s2['state']]

##### Merged all the data

In [ ]:
#events_s2['device_id']=events_s2['device_id'].map(lambda x: '{:.0f}'.format(x)).astype(str)
mob_devices['device_id']=mob_devices['device_id'].astype('str')
c['device_id']=c['device_id'].astype('str')

In [ ]:
merg1=pd.merge(final_data,mob_devices,how='left',on='device_id')
tot_data=pd.merge(merg1,c, how='left',on='device_id')

In [ ]:
tot_data.isnull().sum()

In [ ]:
tot_data.drop(['longitude_x','latitude_x','longitude_y','latitude_y','state_latlong'],axis=1,inplace=True)

In [ ]:
tot_data=tot_data[tot_data['state'].isin(['MadhyaPradesh','Chhattisgarh','Uttaranchal','JammuandKashmir','Goa','Nagaland'])].reset_index()

In [ ]:
tot_data['state'].value_counts()

##### Analysis after merging

In [ ]:
print("    Missing per field \n", tot_data.isnull().sum())
print("----------------------------------------------")
print("Number of Device IDs with no info from user and device tables -",tot_data[tot_data['phone_brand'].isnull()]['device_id'].nunique())
print("----------------------------------------------")
print("Number of Device IDs with info from user and device tables -",tot_data[tot_data['phone_brand'].notnull()]['device_id'].nunique())
print("----------------------------------------------")
notmissing_device=tot_data[tot_data['phone_brand'].notnull()]['device_id']
missing_device=tot_data[tot_data['phone_brand'].isnull()]['device_id']
a=set(notmissing_device)
b=set(missing_device)
print("DeviceIds missing if is present in not missing Device IDs- ",len(a.intersection(b)))


##### 1. Distribution of Users(device_id) across States.

In [ ]:
data=tot_data[['device_id','state']].drop_duplicates()

plt.figure(figsize=(12,6))

sns.countplot(x='state',data=data,palette='muted').set_title('Distribution of Users across States')
plt.xticks(rotation=90)
plt.show()

##### 2. Distribution of Users across Phone Brands(Consider only 10 Most used Phone Brands).



In [ ]:
pd.set_option('display.max_rows',10000)
data1=pd.DataFrame(tot_data.groupby(['eng_brand'])['device_id'].nunique()).sort_values('device_id',ascending=False)[:10].reset_index()
data1['perc_dist']=(data1['device_id']/sum(data1['device_id'])*100)




In [ ]:
data1

In [ ]:
data2=data1
plt.figure(figsize=(12,8))
ax=sns.barplot(x='eng_brand',y='perc_dist',data=data2,palette='muted').set_title('Distribution of Users across Phone Brands')
plt.xticks(rotation=90)
plt.show()

##### 3. Distribution of Users across Gender

##### 4. Distribution of Users across Age Segments

In [ ]:
#notnull_data=tot_data[tot_data['phone_brand'].notnull()].reset_index()
l=[]
for i in range(len(tot_data)):
    if tot_data['age'][i]<= 22:
        l.append('<22')
    elif 22 < tot_data['age'][i]<= 26:
        l.append('23-26')
    elif 26 < tot_data['age'][i]<=28:
        l.append('27-28')
    elif 28 < tot_data['age'][i]<=31:
        l.append('29-31')
    elif 31 < tot_data['age'][i]<=38:
        l.append('31-38')
    else:
        l.append('39+')

tot_data['age_grp']=l

data2=tot_data[['gender','device_id']].drop_duplicates()

data3=tot_data[['age_grp','device_id']].drop_duplicates()


fig,ax=plt.subplots(1,2,figsize=(12,8))

sns.countplot(x='gender',data=data2,palette='muted',ax=ax[0]).set_title('Distribution of Users across Gender.')
#sns.countplot(x='age',data=user_Data,palette='ch:.25',ax=ax[1]).set_title('Age distribution')
#sns.countplot(x='group',order=['M22-','M23-26','M27-28','M29-31','M32-38','M39+','F23-','F24-26','F27-28','F29-32','F33-42','F43+'],data=user_Data,palette='copper',ax=ax[1]).set_title('Group Distribution')
sns.countplot(x='age_grp',data=data3,palette='Set1',ax=ax[1]).set_title('Distribution of Users across Age Segments.')



total=[len(data2),len(data3)]
for i in range(2):
    ax[i].legend(loc='upper right', frameon=False)
    for tick in ax[i].get_xticklabels():
        tick.set_rotation(45)
    for p in ax[i].patches:
        height = p.get_height()
        ax[i].text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}'.format(height/total[i]*100),
                ha="center") 

        

plt.show()



In [ ]:
top_brands=pd.DataFrame(tot_data.groupby(['eng_brand'])['device_id'].nunique()).sort_values('device_id',ascending=False)[:10].reset_index()['eng_brand']

In [ ]:
data_top10brand=tot_data[tot_data['eng_brand'].isin(top_brands)]
data4=data_top10brand[['device_id','age_grp','state','gender','eng_brand']].drop_duplicates()

##### 5. Distribution of Phone Brands(Consider only 10 Most used Phone Brands) for each Age Segment, State, Gender

In [ ]:
data5=pd.DataFrame(data_top10brand.groupby(['age_grp','state','gender','eng_brand'])['device_id'].nunique()).reset_index()

In [ ]:
import plotly.express as px

fig = px.treemap(data5, path=['age_grp','state','gender','eng_brand'],values='device_id',width=1000, height=600)
fig.data[0].textinfo = 'label+text+value'
fig.show()

##### 6. Distribution of Gender for each State, Age Segment and Phone Brand(Consider only 10 Most used Phone Brands).


In [ ]:
data6=pd.DataFrame(data_top10brand.groupby(['state','age_grp','eng_brand','gender'])['device_id'].nunique()).reset_index()

In [ ]:

fig = px.treemap(data6, path=['state','age_grp','eng_brand','gender'],values='device_id',width=1000, height=600)
fig.data[0].textinfo = 'label+text+value'
fig.show()

##### 7. Distribution of Age Segments for each State, Gender and Phone Brand(Consider only 10 Most used Phone Brands).

In [ ]:
data7=pd.DataFrame(data_top10brand.groupby(['state','gender','eng_brand','age_grp'])['device_id'].nunique()).reset_index()

In [ ]:
fig = px.treemap(data7, path=['state','gender','eng_brand','age_grp'],values='device_id',width=1000, height=600)
fig.data[0].textinfo = 'label+text+value'
fig.show()

##### 8. Hourly distribution of Phone Calls.

In [ ]:
tot_data.timestamp=pd.to_datetime(tot_data.timestamp)
date_data=tot_data[['event_id','timestamp']].set_index('timestamp')
date_data_hourly=date_data.resample('H').count().reset_index()

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.lineplot(x="timestamp", y="event_id", data=date_data_hourly).set_title('Hourly distribution of Phone Calls')
plt.show()

In [ ]:
date_data_split=date_data_hourly
date_data_split['timestamp']=date_data_split['timestamp'].astype(str)
date_data_split['date']=date_data_split['timestamp'].apply(lambda x:x.split(' ')[0])
date_data_split['time']=date_data_split['timestamp'].apply(lambda x:x.split(' ')[1])
date_data_split.drop(['timestamp'],axis=1,inplace=True)

In [ ]:
date_data_split['time']=date_data_split['time'].apply(lambda x:x.rsplit(':',1)[0])

In [ ]:
date_data_split=date_data_split.sort_values(["time"])
plt.figure(figsize=(20,6))
ax = sns.lineplot(x="time", y="event_id",hue="date",sort= True,data=date_data_split).set_title('Hourly distribution of Phone Calls')

plt.xticks(rotation=90)
plt.show()

##### 9. Plot the Users on the Map using any suitable package.

In [ ]:
data1=tot_data[['latitude','longitude']].drop_duplicates()

In [ ]:
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap

map_osm = Map(location=[28.730140, 77.225676], zoom_start=6)


#data1.apply(lambda row:Marker(location=[row["latitude"], row["longitude"]], 
                                             # radius=1)
                                             #.add_to(map_osm), axis=1)

hm_wide = HeatMap( list(zip(data1.latitude.values, data1.longitude.values)),
                     min_opacity=0.2,
                     radius=17, blur=15,
                     max_zoom=6
                 )
map_osm.add_child(hm_wide)

map_osm.save('heatmap_loc.html')

In [ ]:

map_osm = Map(location=[28.730140, 77.225676], zoom_start=6)


data1.apply(lambda row:Marker(location=[row["latitude"], row["longitude"]], 
                                             radius=1)
                                             .add_to(map_osm), axis=1)


map_osm.save('locationmap.html')

In [ ]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster

map_points = Map(location=[28.730140, 77.225676], zoom_start=4)
# These two lines should create FastMarkerClusters
FastMarkerCluster(data=list(zip(data1.latitude.values, data1.longitude.values))).add_to(map_points )
folium.LayerControl().add_to(map_points )

for index, row in data1.iterrows():

    # generate the popup message that is shown on click.
    popup_text = "{}<br> latitude: {:,}<br> longitude: {:,}"
    popup_text = popup_text.format(
                      index,
                      row["latitude"],
                      row["longitude"]
                      )

    folium.CircleMarker(location=(row["latitude"],
                                  row["longitude"]),
                        radius= 4,
                        color="#007849",
                        popup=popup_text,
                        fill=False).add_to(map_points)
map_points.save('latlong_count.html')
